# Project Group - 25

Members: Yun-An LIN (Jackie), Rohan Menezes, John Kuttikat, Muhammad Rizki Ziarieputra (Kiki), Ian Trout 

Student numbers: 5841682, 5850908, 5765382, 5848113, 5851483

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

Vessel time spent in ports by country before and during COVID--an analysis by ship category showing the impacts of COVID

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: coding, background research, conceptualisation

**Author 2**:coding, visualisation

**Author 3**: coding, data analysis
    
**Author 4**:coding, data modelling

**Author 5**: coding, visualisation

# Data Used

----Covid data (https://data.humdata.org/dataset/coronavirus-covid-19-cases-and-deaths) 

----Port data (https://unctadstat.unctad.org/wds/TableViewer/tableView.aspx?ReportId=170027)

----total cargo loaded/unloaded by region from 1970 to 2020 (https://www.kaggle.com/datasets/illiaparfeniuk/maritime-trading-volumes)

----Total amount of goods imported and exported by ship per EU country(https://ec.europa.eu/eurostat/databrowser/view/ttr00009/default/map?lang=en)

----Container Port Performance Index from 2020 to 2021 by World Bank Group(https://thedocs.worldbank.org/en/doc/66e3aa5c3be4647addd01845ce353992-0190062022/original/Container-Port-Performance-Index-2021.pdf)

# Data Pipeline

take only the last 6 months of each year (limitation of the maritime data): 
    
convert the maritime data:
    
    1) to a common volume 
    
    2) calculate the average volume for all cargo types 
    
    3) consolidate the data into regions of the world. 

convert COVID cases: 
    1) calculate the average vaccination cases per country that has reported it 
    2) calculate the average COVID cases per country for the last 6 months of every year (July to December) 
    
Analyze port call times for 2018, 2019, compared to 2020 to see the difference with COVID.

---calculate the differences 

Compare the 2020 and 2021 port call times to see if improvements have been made or if port calls are still slow. 

Visually show the change in port call times by region of the world by year. 



first, we will import the necessary libraries



In [1]:
import pandas as pd
import chardet
from plotly.offline import init_notebook_mode
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
import itertools
import geopandas as gpd
# import geoplot
# import geoplot.crs as gcrs
import math
import scipy
from scipy.signal import find_peaks
from datetime import datetime

init_notebook_mode(connected=True)
pio.renderers.default = "plotly_mimetype+notebook"

## Part I

First, We're going to import and combine dataframes of the four types of data I found:

Covid data from the WHO on country level, giving cases, hospitalizations and casualties per day (absolute and cumulative)

Port data from UNICSTAT on a country level, giving tonnage, median time in port, and other information (from 2018 to 2022) 

Port peformance index data for several ports within a country (data ranging from 2020 to 2021) 

GeoJSON file of all the countries in the world


We're starting off with the port and the geocoding datasets.

In [34]:
file_path = r"/Users/iantrout/TIL6022-group_project/Data/Maritime data/US_PortCalls_S_ST202209220924_v1.csv"
with open(file_path, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [35]:
# Importing and Touching-up the Data

df_ports = pd.read_csv(file_path,encoding='utf-8')
df_ports['Period Label'] = df_ports['Period Label'].str.replace('   ','-')
df_ports = df_ports.drop(columns=['Period', 'Frequency', 'Frequency Label', 'Economy', 
                                      'CommercialMarket', 'Median time in port (days) Footnote',
                                      'Average age of vessels Footnote', 'Average size (GT) of vessels Footnote',
                                      'Maximum size (GT) of vessels Footnote', 'Average cargo carrying capacity (dwt) per vessel Footnote',
                                      'Maximum cargo carrying capacity (dwt) of vessels Footnote','Average container carrying capacity (TEU) per container ship Footnote',
                                      'Maximum container carrying capacity (TEU) of container ships Footnote'])
df_ports.rename(columns = {'Economy Label': 'country', 'CommercialMarket Label': 'Vessel_Type', }, inplace=True)
date_change=[]
for row in df_ports['Period Label']:
    if row == 'S1-2018' :   date_change.append('2018-07-31')
    elif row == 'S2-2018':   date_change.append('2019-01-31')
    elif row == 'S1-2019':  date_change.append('2019-07-31')
    elif row == 'S2-2019':  date_change.append('2020-01-31')
    elif row == 'S1-2020':  date_change.append('2020-07-31')
    elif row == 'S2-2020':  date_change.append('2021-01-31')
    elif row == 'S1-2021':  date_change.append('2021-07-31')
    elif row == 'S2-2021':  date_change.append('2022-01-31')
    elif row == 'S1-2022':  date_change.append('2022-07-31')
    else:           date_change.append('Not_Rated')

df_ports = df_ports.drop(columns=['Period Label'])
df_ports['date'] = date_change
df_ports
df_ports.head()

,Year,country,Vessel_Type,Median time in port (days),Average age of vessels,Average size (GT) of vessels,Maximum size (GT) of vessels,Average cargo carrying capacity (dwt) per vessel,Maximum cargo carrying capacity (dwt) of vessels,Average container carrying capacity (TEU) per container ship,Maximum container carrying capacity (TEU) of container ships,date
0,2018,World,All ships,0.97,18,15222,234006,24074.0,441561.0,3526.0,21413.0,2018-07-31
1,2018,World,Passenger ships,NaN,21,8978,228081,NaN,NaN,NaN,NaN,2018-07-31
2,2018,World,Liquid bulk carriers,0.94,13,15470,234006,26871.0,441561.0,NaN,NaN,2018-07-31
3,2018,World,Container ships,0.69,13,38405,217673,NaN,NaN,3526.0,21413.0,2018-07-31
4,2018,World,Dry breakbulk carriers,1.12,19,5455,91784,7413.0,138743.0,NaN,NaN,2018-07-31


In [36]:
# df_combined=pd.merge(df_new,df_ports_world,on=['country','date'])
# df_combined.head()

df_combined = pd.merge(df_ports, df_new, on=['country','date'], how='outer')
df_combined = df_combined.drop(['Unnamed: 0', 'cumulative_cases'], axis=1)
df_combined

NameError: name 'df_new' is not defined

In [37]:
# Importing and Touching-up the Port Calls Data

file_path3 = r"/Users/iantrout/TIL6022-group_project/Data/Maritime data/US_PortCallsArrivals_S_ST202209220927_v1.csv"
with open(file_path3, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

df_port_calls = pd.read_csv(file_path3,encoding='utf-8')
df_port_calls['Period Label'] = df_port_calls['Period Label'].str.replace('   ','-')
df_port_calls = df_port_calls.drop(columns=['Period', 'Frequency', 'Frequency Label', 'Economy', 
                                       'CommercialMarket', 'Number of port calls Footnote',])
df_port_calls.rename(columns = {'Economy Label': 'country', 'CommercialMarket Label': 'Vessel_Type', }, inplace=True)
date_change=[]
for row in df_port_calls['Period Label']:
    if row == 'S1-2018' :   date_change.append('2018-07-31')
    elif row == 'S2-2018':   date_change.append('2019-01-31')
    elif row == 'S1-2019':  date_change.append('2019-07-31')
    elif row == 'S2-2019':  date_change.append('2020-01-31')
    elif row == 'S1-2020':  date_change.append('2020-07-31')
    elif row == 'S2-2020':  date_change.append('2021-01-31')
    elif row == 'S1-2021':  date_change.append('2021-07-31')
    elif row == 'S2-2021':  date_change.append('2022-01-31')
    elif row == 'S1-2022':  date_change.append('2022-07-31')
    else:           date_change.append('Not_Rated')

df_port_calls = df_port_calls.drop(columns=['Period Label'])
df_port_calls['date'] = date_change
df_port_calls

,Year,country,Vessel_Type,Number of port calls,date
0,2018,World,All ships,1984908,2018-07-31
1,2018,World,Passenger ships,1053697,2018-07-31
2,2018,World,Liquid bulk carriers,245147,2018-07-31
3,2018,World,Container ships,226063,2018-07-31
4,2018,World,Dry breakbulk carriers,211031,2018-07-31
...,...,...,...,...,...
15697,2022,United Kingdom,Dry breakbulk carriers,7967,2022-07-31
15698,2022,United Kingdom,Dry bulk carriers,956,2022-07-31
15699,2022,United Kingdom,Roll-on/ roll-off ships,7983,2022-07-31
15700,2022,United Kingdom,Liquefied petroleum gas carriers,632,2022-07-31


In [11]:
df_combined2 = pd.merge(df_port_calls, df_new, on=['country','date'], how='outer')
df_combined2 = df_combined2.drop(['Unnamed: 0', 'cumulative_cases'], axis=1)
df_combined2

NameError: name 'df_port_calls' is not defined

In [49]:
# Filtering the maritime data (Median time in port (days)) from 2020 to 2021
file_time_port_path = "/Users/iantrout/TIL6022-group_project/Data/Maritime data/US_PortCalls_S_ST202209220924_v1.csv"
df_port_time = pd.read_csv(file_time_port_path)
df_port_time = df_port_time.drop(df_port_time[df_port_time["Year"] == 2018].index)
df_port_time = df_port_time.drop(df_port_time[df_port_time["Year"] == 2019].index)
df_port_time = df_port_time.drop(df_port_time[df_port_time["Year"] == 2022].index)



In [50]:
# now merge two dataframes (port time and CPPI)
import chardet
port_path = "/Users/iantrout/TIL6022-group_project/Data/The productivity of the ports/Container-Port-Performance-Index-2021 copy.csv"

port= pd.read_csv(port_path)

# using merge function by setting how='outer'

output1 = pd.merge(df_port_time, port,
                   on=['Economy Label', "Year"],
                   how = "inner")

column_names = ['Port Name', 'CommercialMarket Label']
output1.drop_duplicates(subset=column_names, keep='first', inplace=True)
output1 = output1.drop(output1[output1["CommercialMarket Label"] != "Container ships"].index)
output1 = output1.drop(output1[output1["Economy Label"] == "World"].index)
sort = output1.sort_values("Statistic Approach Rank")

# displaying result
output1.head()

,Period,Period Label,Year,Frequency,Frequency Label,Economy,Economy Label,CommercialMarket,CommercialMarket Label,Median time in port (days),...,Maximum cargo carrying capacity (dwt) of vessels Footnote,Average container carrying capacity (TEU) per container ship,Average container carrying capacity (TEU) per container ship Footnote,Maximum container carrying capacity (TEU) of container ships,Maximum container carrying capacity (TEU) of container ships Footnote,Port Name,Administrative Approach Rank,Total\nPoints,Statistic Approach Rank,IndeX Value
18,2020S01,S1 2020,2020,S,Semi-annual,36,Australia,3,Container ships,1.3083,...,NaN,4653.0,NaN,9572.0,NaN,ACAJUTLA,249,0.640,249,-11.0
19,2020S01,S1 2020,2020,S,Semi-annual,36,Australia,3,Container ships,1.3083,...,NaN,4653.0,NaN,9572.0,NaN,BELL BAY,163,-0.205,163,5.0
20,2020S01,S1 2020,2020,S,Semi-annual,36,Australia,3,Container ships,1.3083,...,NaN,4653.0,NaN,9572.0,NaN,BRISTOL,294,1.462,277,-19.0
21,2020S01,S1 2020,2020,S,Semi-annual,36,Australia,3,Container ships,1.3083,...,NaN,4653.0,NaN,9572.0,NaN,FREETOWN,220,0.350,319,-49.0
22,2020S01,S1 2020,2020,S,Semi-annual,36,Australia,3,Container ships,1.3083,...,NaN,4653.0,NaN,9572.0,NaN,MOJI,111,-0.663,123,11.0


In [51]:
#Data of the number of port calls and filtering them down to just 2020 and 2021
file_path = '/Users/iantrout/TIL6022-group_project/Data/Maritime data/US_PortCallsArrivals_S_ST202209220927_v1.csv'
df_call = pd.read_csv(file_path)
df_call = df_call.drop(df_call[df_call["Year"] == 2018].index)
df_call = df_call.drop(df_call[df_call["Year"] == 2019].index)
df_call = df_call.drop(df_call[df_call["Year"] == 2022].index)
df_call.head()
#path = "C:/Users/Jack/TIL6022-group_project/Data/The productivity of the ports"
#df_call.to_csv('%s number_call.csv'%(path), index = False)

,Period,Period Label,Year,Frequency,Frequency Label,Economy,Economy Label,CommercialMarket,CommercialMarket Label,Number of port calls,Number of port calls Footnote
7007,2020S01,S1 2020,2020,S,Semi-annual,0,World,0,All ships,1863718,NaN
7008,2020S01,S1 2020,2020,S,Semi-annual,0,World,1,Passenger ships,951245,NaN
7009,2020S01,S1 2020,2020,S,Semi-annual,0,World,2,Liquid bulk carriers,243100,NaN
7010,2020S01,S1 2020,2020,S,Semi-annual,0,World,3,Container ships,225347,NaN
7011,2020S01,S1 2020,2020,S,Semi-annual,0,World,4,Dry breakbulk carriers,201473,NaN


In [52]:
# megre two dataframes (the number of port calls and CPPI)
import chardet
port_path = "/Users/iantrout/TIL6022-group_project/Data/The productivity of the ports/Container-Port-Performance-Index-2021 copy.csv"
 
port= pd.read_csv(port_path)
# using merge function by setting how='outer'
output = pd.merge(port, df_call, 
                   on=['Economy Label', "Year"],
                   how = "inner")

column_names = ['Port Name', 'CommercialMarket Label']
output.drop_duplicates(subset=column_names, keep='first', inplace=True)
output = output.drop(output[output["CommercialMarket Label"] != "Container ships"].index)
output = output.drop(output[output["Economy Label"] == "World"].index)

# displaying result
output.head()
#path = "C:/Users/Jack/TIL6022-group_project/Data/The productivity of the ports"
#output.to_csv('%s output_2.csv'%(path), index = False)

,Port Name,Administrative Approach Rank,Total\nPoints,Statistic Approach Rank,IndeX Value,Economy Label,Year,Period,Period Label,Frequency,Frequency Label,Economy,CommercialMarket,CommercialMarket Label,Number of port calls,Number of port calls Footnote
3,AARHUS,44,-2.036,43,43.000,Denmark,2020,2020S01,S1 2020,S,Semi-annual,208,3,Container ships,660,NaN
19,COPENHAGEN,129,-0.471,142,8.000,Denmark,2020,2020S01,S1 2020,S,Semi-annual,208,3,Container ships,660,NaN
35,FREDERICIA,128,-0.493,138,9.000,Denmark,2020,2020S01,S1 2020,S,Semi-annual,208,3,Container ships,660,NaN
51,LAGOS (NIGERIA),358,-188.317,358,-80.295,Denmark,2020,2020S01,S1 2020,S,Semi-annual,208,3,Container ships,660,NaN
67,ACAJUTLA,249,0.640,249,-11.000,Australia,2020,2020S01,S1 2020,S,Semi-annual,36,3,Container ships,1884,NaN


# Part II

we start by understanding how many countries we have data for and for that we will plot a world map for All ship types combined

now we can't only infer information from graphs, so we will calculate the peaks and valleys of the COVID data and the port data to see if there is a match based on serveral values of the port data (avg age of the vessel, average size of the vessel, average time in port)

In [21]:
# file_path2 = r"C:\Users\user\OneDrive - Delft University of Technology\Desktop\TIL\Q1\TIL6022\Final Project\covid_data_new.csv"
# with open(file_path, 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(100000))
# result

In [22]:
df_ports_world = df_ports[df_ports.country == 'World']
df_covid_world = df_new.groupby('date').sum()
df_covid_world = df_covid_world.drop(['2020-01-31','2023-01-31'])

df_combined_world =pd.merge(df_ports_world, df_covid_world, on=['date'], how='outer')
df_combined_world = df_combined_world.drop(['Unnamed: 0', 'cumulative_cases'], axis=1)
df_combined_world

NameError: name 'df_ports' is not defined

In [23]:
# df_combined_world.to_csv(r"C:\Users\user\OneDrive - Delft University of Technology\Desktop\TIL\Q1\TIL6022\Final Project\result.csv")

In [24]:
df_combined_world = df_combined_world[df_combined_world['Vessel_Type'] == 'All ships']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_combined_world['date'], y=df_combined_world['Median time in port (days)'], name="Median time in port (days)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_combined_world['date'], y=df_combined_world['new_cases'], name="New covid cases"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="World - all vessel type"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Median time in port (days)", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> New covid cases", secondary_y=True)

fig.show()

NameError: name 'df_combined_world' is not defined

In [25]:
df_port_calls_world = df_port_calls[df_port_calls.country == 'World']
df_covid_world = df_new.groupby('date').sum()
df_covid_world = df_covid_world.drop(['2020-01-31','2023-01-31'])

df_combined_world_calls = pd.merge(df_port_calls_world, df_covid_world, on=['date'], how='outer')
df_combined_world_calls = df_combined_world_calls.drop(['Unnamed: 0', 'cumulative_cases'], axis=1)
df_combined_world_calls

NameError: name 'df_port_calls' is not defined

In [26]:
df_combined_world_calls = df_combined_world_calls[df_combined_world_calls['Vessel_Type'] == 'All ships']

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_combined_world_calls['date'], y=df_combined_world_calls['Number of port calls'], name="Number of port calls"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_combined_world_calls['date'], y=df_combined_world_calls['new_cases'], name="New covid cases"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="World - all vessel type"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> Number of port calls", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> New covid cases", secondary_y=True)

fig.show()

NameError: name 'df_combined_world_calls' is not defined

## Part III - Data visualisation

For this last part, we're going to visually show the effect that COVID had on vessel times so that users can see how ports have been impacted by COVID and thus has also impacted the logistics system as a whole by: 

We're going to look at regions and look at the semi annual trend by vessel type 

pie chart showing the proportions of the commodity shipped

World map showing the change in port call times over the years 

Comparing covid high periods vs low periods with port call times 

Interpreting the results 

First, We show our variables for this part.

We want to show the COVID data with the port time (worldwide)

In [27]:
# First, I'll make a graph of all covid data in the world
fig_5 = go.Figure()

x1 = df_ports['Period Label']
y1 = df_ports['Median time in port (days)']
fig_5.add_trace(go.Scatter(x=x1,y=y1, name=activity_5))


fig_5.update_layout(title='Covid data in world')
fig_5.show()

NameError: name 'df_ports' is not defined

Now we will show over the years from 2018, the number of port calls by region

In [47]:
# CPPI by bubble chart with size = number of port calls
size = output["Number of port calls"]
fig = px.scatter(output, x="Port Name", y="Statistic Approach Rank",
                 size="Number of port calls", color="Economy Label", hover_data = ["Port Name"],
                 width = 2000, height = 500
                )
fig.update_xaxes(categoryorder="total ascending")

fig.show()

In [46]:
fig = px.bar(output1, x="Statistic Approach Rank", y="Median time in port (days)",
                 color="Economy Label", hover_data = ["Port Name", "Year"],
                 width = 2000, height = 500
            )
fig.update_xaxes(categoryorder="total ascending")

fig.show()

In [48]:
# The comparison of each port for 2020 and 2021 (there is erro and still working on)
import plotly.graph_objects as go
x = port["Economy Label"]
y = port["IndeX Value"]
year_2020 = port.query("Year==2020")
y1 = year_2020.groupby("IndeX Value").sum()

year_2021 = port.query("Year==2021")
y2 = year_2021.groupby("IndeX Value").sum()


fig = go.Figure()
fig.add_trace(go.Bar(
    x= x,
    y= y1,
    name = "2020",
    marker_color='blue'
))
fig.add_trace(go.Bar(
    x= x,
    y= y2,
    name='2021',
    marker_color='green'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()